In [1]:
import re
from ufal.udpipe import Model, Pipeline, ProcessingError
import pandas as pd 
from conllu import parse

model = Model.load('G:/Softwares/urdu-udtb-ud-2.3-181115.udpipe')
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
error = ProcessingError()

In [2]:
from __future__ import unicode_literals
# -*- coding: utf-8 -*-
STOP_WORDS = """
 ابھی از 
 اس استعمال اسی اسے البتہ الف ان اندر اور اوپر اکثر اگر
 اگرچہ اگلے ایسا ایسی ایسے اے بار بارے باوجود باہر بظاہر بعض بغیر بلکہ بن
 بھر بھریں بھی بہت بے تا تاکہ تب تحت تر تمام
 تو تک جب سوال 
 جبکہ جو حالانکہ حالاں خلاف
 خود سے میں چنانچہ دیر ذریعے تھا هے ایک آپ
 سا ساتھ سامنے سب تھی تھے
 سکا سکتا سکتے سی سے شان شاید صرف صورت ضرورت ضروری طرح طرف طور علاوہ عین غیر 
 لہذا لیکن لیں لیے لے مجھ مجھے مزید مقابلے مل مکمل مگر 
 نا نہ نہیں نیچے واقعی والا والوں والی والے وجہ وغیرہ وہ وہاں وہی وہیں وی ویسے پاس
 پایا پر پوری پھر پیچھے چونکہ چکی
 ڈالے کئے کافی کبھی کسی کم 
 کوئی کچھ کہ کہا کہہ کہیں کہے کیونکہ کیے کے گئی
 گئے گا گویا گی گے ہاں ہر ہمیشہ ہو ہوئی ہوئیں ہوئے ہوا ہوتا
 ہوتی ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیں ہے یا یات یعنی یقینا یہ یہاں یہی یہیں ھیں
  حضرت ہوگی رض
 کا کو کی نے ہوتو پہلے کر
 اپنا حتى كہ ميں لئیے  
 بعد ارادہ افاقہ آخر
""".split()

In [3]:
from gensim.models import word2vec
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('urduvec_140M_100K_300d.bin', binary=True)

C:\Users\admin\Anaconda3\envs\islamic qa\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## To get row count only

In [155]:
import pyodbc
from gensim.models import Word2Vec
import sys
import time
start = time.time()

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=G:\Softwares\EQWH-V6.4-190407\EasyHadees - Copy.mdb;')
cursor = conn.cursor()
targetcount = 100
cursor.execute("select count(*) from ahadith where urdutext is not null and urdutext like '%وضو%'")

count = cursor.fetchone()[0]
print(count)
cursor.close()
conn.close()

292


In [4]:
import pyodbc
from gensim.models import Word2Vec
import sys
import time
import numpy as np

no_of_docs_cache = {}
idf_cache = {}
term_freq_doc_cache = {}

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=G:\Softwares\EQWH-V6.4-190407\EasyHadees - Copy.mdb;')
cursor = conn.cursor()
cursor.execute("select id, urdutext from ahadith where urdutext is not null and urdutext like '%وضو%'")

Y = []
corpus = []
lens_of_d= []
for row in cursor.fetchall():
    hadith = row[1].replace('صلی اللہ علیہ وآلہ وسلم','')    
    corpus.append(hadith)
    lens_of_d.append(len(hadith.split()))
    Y.append(row[0])
cursor.close()
conn.close()

avg_docs_len = np.mean(lens_of_d, axis=0)

In [5]:
#from sklearn.feature_extraction.text import TfidfVectorizer as vectorizer
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words=STOP_WORDS).fit(corpus)
bag_of_words = vec.transform(corpus)
vocab = vec.vocabulary_.items()

# Ranking (okapi BM25)

![title](img/43e5c609557364f7836b6b2f4cd8ea41deb86a96.svg)

where

![title](img/c652b6871ce4872c8e924ff0f806bc8b06dc94ed.svg)

In [6]:
def term_freq_doc(myword,doc_index):
    if myword not in vec.vocabulary_:
        return 0
    
    qindex = vec.vocabulary_[myword]
    sm = None
    if doc_index in term_freq_doc_cache:
        sm = term_freq_doc_cache[doc_index]
    else:
        #sm = bag_of_words[doc_index].sum(axis=0)
        sm = bag_of_words[doc_index].toarray()[0]
        term_freq_doc_cache[doc_index] = sm
    freq = sm[qindex]
    return freq

In [7]:
bag_of_words_arr = bag_of_words.toarray()

def no_of_docs(q):
    if q in no_of_docs_cache:
        return no_of_docs_cache[q]
    
    if q not in vec.vocabulary_:
        return 0
    
    qindex = vec.vocabulary_[q]
    
    no_of_ds = 0
    
    for bw in bag_of_words_arr:
        if bw[qindex] > 0:
            no_of_ds += 1
    
    no_of_docs_cache[q] = no_of_ds;
    return no_of_ds        

In [8]:
import math

def idf(q):
    if q in idf_cache:
        return idf_cache[q]
    
    nq = no_of_docs(q)
    ans = math.log10(  (len(Y)- nq + 0.5) / (nq+ 0.5)  )
    idf_cache[q] = ans;
    return ans

def bm25_score(qwords,doc_index):
    score = 0;
    for qw in qwords:
        #start = time.time()
        tfd = term_freq_doc(qw, doc_index)
        score += idf(qw) * ( (tfd * (1.2 + 1))/ (tfd + 1.2 * (1-0.75 + 0.75 * (lens_of_d[doc_index] / avg_docs_len))) + 1.0)
        #end = time.time()
        #print(" time taken: "+str(end - start)+ ", for word: " + qw + ", for doc: " + str(doc_index))
    return score

In [49]:
l = list(vocab)
def vocab_check(w):
    for i in range(0,len(l)):
        if l[i][0] == w:
            return True
            break
    return False

In [21]:
question = 'آپ کس طرح وضو کرتے تھے'
question_arr = question.split()

query_words = []
for qs in question_arr:
    query_words.append(qs)
    query_words.append(qs)
    query_words.append(qs)
for question_word in question_arr:
    alltop = model.most_similar(positive=question_word, topn=5)
    for key, val in [item for item in alltop if item[1] > 0.51 and item[0] in vec.vocabulary_]:
        query_words.append(key)

In [22]:
doc_scores = []
for i in range(0, len(Y)):
    start = time.time()
    score = bm25_score(query_words, i)
    end = time.time()
    #print(" time taken: "+str(end - start)+ ", for doc: " + str(i))
    doc_scores.append({'docid':i, 'score': score})

doc_scores = sorted(doc_scores, key = lambda i: i['score'],reverse=True) 

In [23]:
print(doc_scores[:10])
#print(corpus[doc_scores[0]['docid']])
for i in range(0,5):
    print(corpus[doc_scores[i]['docid']])
    print()

[{'docid': 6683, 'score': 66.660352136007}, {'docid': 1515, 'score': 65.611704482416}, {'docid': 5217, 'score': 65.53605700179644}, {'docid': 4839, 'score': 64.66447511788445}, {'docid': 814, 'score': 64.46825785145239}, {'docid': 654, 'score': 64.35485613847231}, {'docid': 52, 'score': 64.25538830574476}, {'docid': 810, 'score': 63.842699561133465}, {'docid': 3953, 'score': 63.78099278925216}, {'docid': 6074, 'score': 63.753560825739136}]
864 ۔ حضرت علی بن ابوطالب (رض) بیان کرتے : میں نے نبی کریم () سے اس پٹی کے بارے میں دریافت کیا جو کسی زخم وغیرہ پر باندھی جاتی ہے ایساشخص کس طرح وضو کرسکتا ہے ، اگر وہ جنبی ہوجائے تو وہ غسل کس طرح کرے گا ؟ تو نبی کریم () نے فرمایا ایساشخص جنابت کی حالت میں پانی کے ذریعے اس پر مسح کرلے گا، پھر میں نے عرض کی ، اگر سردی ہو یا اپنی جان کا اندیشہ ہو ؟ تو نبی کریم () نے فرمایا وہ اپنے پورے جسم پر مسح کرلے گا، پھر نبی کریم () نے یہ آیت تلاوت کی :
'' تم اپنے آپ کو قتل نہ کرو بیشک اللہ تعالیٰ تمہارے بارے میں رحم کرنے والا ہے۔ (تو نبی کریم () نے فرمایا) ایسے ش

In [20]:
print(query_words)

['آپ', 'آپ', 'آپ', 'کا', 'کا', 'کا', 'وضو', 'وضو', 'وضو', 'کا', 'کا', 'کا', 'طریقہ', 'طریقہ', 'طریقہ', 'کیا', 'کیا', 'کیا', 'تھا', 'تھا', 'تھا', 'غسل', 'جنابت', 'تیمم', 'مسواک', 'استنجاء', 'طریق', 'طریقے', 'کرایا', 'کردیا', 'کروایا']
